In [1]:
##### INITIALIZATION

###INITIALIZATION

%matplotlib inline
import pylab as pl
import numpy as np
import time

from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import make_scorer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import SpectralClustering, KMeans

from pykernels.regular import Tanimoto        #Insight from Mayr et al. (2016); further knowledge from Ralaivola et al. (2005) and Bajusz et al. (2015)


print("Tox21 Model initiated...")
print()

X = []
samples_X = []

f = open("data.csv","r")

for i,line in enumerate(f):
    if i == 0:
        continue
    sv = line.strip().split(",")
    samples_X.append(sv[0])
    
    row = []
    for element in sv[1:]:
        row.append(float(element))
    X.append(row)
    
f.close()

X = np.array(X)
samples_X = np.array(samples_X)
samples_X = np.char.strip(samples_X, '"')   #dont need " marks on each chemical component name 

print("DATA")
print("Number of samples:\t\t%d" % X.shape[0])
print("Number of features:\t\t%d" % X.shape[1])
missing_values = (np.isnan(X)).sum()
print("Number of missing values:\t%d" % missing_values)


y = []
samples_y = []

f = open("toxicity_labels.csv","r")

for i,line in enumerate(f):
    if i == 0:
        continue
    sv = line.strip().split(",")
    
    row = []
    for element in sv[1:]:
        row.append(element)
    samples_y.append(sv[0])
    y.append(row)
    
f.close()

f = open("unknown_data.csv","r")

unknowns = [] 
sample_ids_unknowns = []

for line in f:
    sv = line.strip().split(",")
    sample_ids_unknowns.append(sv[0])
    
    row = []
    for element in sv[1:]:
        row.append(element)
    unknowns.append(row)

sample_ids_unknowns = np.array(sample_ids_unknowns)
sample_ids_unknowns = sample_ids_unknowns[1:]

unknowns = np.array(unknowns)
unknowns = unknowns[1:,:]

f.close()

y = (np.where(np.array(y) == 'NA', np.NaN, np.array(y))).astype(np.float)          ##### WOW this was tough. found answer: https://stackoverflow.com/questions/3877209/how-to-convert-an-array-of-strings-to-an-array-of-floats-in-numpy
samples_y = np.array(samples_y)

print()
print("TOXICITY LABELS")
print("Number of samples:\t\t%d" % y.shape[0])
print("Number of toxicity pathways:\t%d" % y.shape[1])
NA_values = np.count_nonzero(np.isnan(y))
print("Number of missing values:\t%d" % NA_values)
NA_values_AhR = np.count_nonzero(np.isnan(y[:,0]))
print("Number of missing AhR values:\t%d" % NA_values_AhR)
print()
print("UNKNOWNS")
print("Number of samples:\t\t%d" % unknowns.shape[0])
print("Number of features:\t\t%d" % unknowns.shape[1])

Tox21 Model initiated...

DATA
Number of samples:		12060
Number of features:		801
Number of missing values:	0

TOXICITY LABELS
Number of samples:		12060
Number of toxicity pathways:	12
Number of missing values:	43238
Number of missing AhR values:	3619

UNKNOWNS
Number of samples:		610
Number of features:		801


In [2]:
##### PREPROCESSING

print("Beginning preprocessing...")
print()


###MISSING DATA: IMPUTE OFF FEATURES, INITIAL STEPS

y = y[:,0]                                 #taking only 1st column of y 
y = y[:, np.newaxis]                       #reshape y into 2D array

na_AhR = np.count_nonzero(np.isnan(y[:,0])) #calculating the number of NAN in first column
no_AhR = np.count_nonzero(y[:,0])           #calculating the number of NAN and 1 in first column
count_AhR = no_AhR - na_AhR
percent_AhR = count_AhR/(12060 - na_AhR)    #percentage of 1's in first column of labels

###PREPROCESSING

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.15,random_state=1)       #Initially did split later; found insight here: https://datascience.stackexchange.com/questions/53138/which-comes-first-multiple-imputation-splitting-into-train-test-or-standardiz
print("Train-Test split results:")
print("Shape of training data:", X_train.shape)
print("Shape of testing data:", X_test.shape)
print()

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)
unknowns = scaler.transform(unknowns)


print("Standardization completed")
print()

#kmeans = KMeans(n_clusters=2, random_state=0)                     ##well this is WRONG AND BAD
#X_train = kmeans.fit_transform(X_train)
#X_test = kmeans.transform(X_test)
#unknowns = kmeans.transform(unknowns)
#print("Clustering completed.")
#print()

###MISSING DATA: IMPUTE OFF FEATURES, FINAL STEPS

imputer = KNNImputer(missing_values=np.nan, n_neighbors=3)            #selected based on Beretta & Santaniello (2016); checked against other options as well.

X_train = np.append(X_train, y_train, axis=1)      #append X_train and y_train into one array 
X_test = np.append(X_test, y_test, axis=1)         #append X_test and y_test into one array

X_train = imputer.fit_transform(X_train)           #imputation on training data
X_test = imputer.transform(X_test)                 #imputation on testing data

y_train = X_train[:, 801]                     #separate into X_train, X_test, y_train, y_test 
y_test = X_test[:, 801]
X_train = X_train[:, :801]
X_test = X_test[:, :801]

y_train = np.around(y_train, decimals=0)
y_test = np.around(y_test, decimals=0)


no_AhR_test = np.count_nonzero(y_test)
no_AhR_train = np.count_nonzero(y_train)
count_AhR_impute = no_AhR_test + no_AhR_train
percent_AhR_impute = count_AhR_impute/12060

print("Imputation completed.")
print("Initial # of samples with AhR = 1: ", count_AhR)
print("Final # of samples with AhR = 1: ", count_AhR_impute)
print()
print("Initial % of samples with AhR = 1:", percent_AhR)
print("Final % of samples with AhR = 1:", percent_AhR_impute)
print()

###PCA

pca = PCA(n_components=0.99)                            #Gained insight from Nguyen & Holmes (2019)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
unknowns = pca.transform(unknowns)
print("PCA results:")
print(X_train.shape[1], "features explain 99% of variance")
print("New shape of training data:", X_train.shape)
print("New shape of testing data:", X_test.shape)
print()

Beginning preprocessing...

Train-Test split results:
Shape of training data: (10251, 801)
Shape of testing data: (1809, 801)

Standardization completed

Imputation completed.
Initial # of samples with AhR = 1:  981
Final # of samples with AhR = 1:  1369

Initial % of samples with AhR = 1: 0.11621845752872882
Final % of samples with AhR = 1: 0.11351575456053067

PCA results:
249 features explain 99% of variance
New shape of training data: (10251, 249)
New shape of testing data: (1809, 249)



In [3]:
############# ALGORITHM 1: KNN

print("KNN model training")
print()

cv = StratifiedKFold(n_splits=5)                    

best_mcc_val = []
best_mcc_test = []
best_auc_test = []
best_precision_test = []
best_recall_test = []
best_neighbors = []

for train_index, test_index in cv.split(X_train,y_train):
    X_train_kn = X_train[train_index]
    X_test_kn = X_train[test_index]
    y_train_kn = y_train[train_index]
    y_test_kn = y_train[test_index]
    
    #Create train-test split für hyperparameter optimization
    X_subtrain, X_val, y_subtrain, y_val = train_test_split(X_train_kn, y_train_kn, test_size = 0.2)
    neighbors = np.arange(1,51)
    
    mcc_train = [] 
    mcc_val = []

    for k in neighbors:
        knn_model = KNeighborsClassifier(n_neighbors=k,metric="minkowski",p=2)
        knn_model.fit(X_subtrain, y_subtrain) 
        predictions_training = knn_model.predict(X_subtrain) 
        predictions_testing = knn_model.predict(X_val) 
    
        #Store Accuracy
        mcc_train.append(metrics.matthews_corrcoef(y_subtrain,predictions_training))
        mcc_val.append(metrics.matthews_corrcoef(y_val,predictions_testing))
    
    #Store information to best model in line-search
    best_k = neighbors[np.argmax(mcc_val)]
    best_neighbors.append(best_k)
    best_mcc_val.append(mcc_val[np.argmax(mcc_val)])
    
    #retrain Model with best k and predict on test data
    knn_model = KNeighborsClassifier(n_neighbors=best_k,metric="minkowski",p=2)
    knn_model.fit(X_train_kn, y_train_kn) 
    y_prediction_knn = knn_model.predict(X_test)
    y_prediction_prob_knn = knn_model.predict_proba(X_test)[:,1]
    test_mcc = metrics.matthews_corrcoef(y_test,y_prediction_knn)
    best_mcc_test.append(test_mcc)
    test_auc = metrics.roc_auc_score(y_test, y_prediction_prob_knn)  #we used prediction_prob function to get the probability of result in between 0 and 1
    best_auc_test.append(test_auc)
    best_precision_test.append(metrics.precision_score(y_test,y_prediction_knn))
    best_recall_test.append(metrics.recall_score(y_test,y_prediction_knn))

    
best_neighbors = np.array(best_neighbors)
best_mcc_val = np.array(best_mcc_val)
best_mcc_test = np.array(best_mcc_test)
best_auc_test = np.array(best_auc_test)
best_recall_test = np.array(best_recall_test)
best_precision_test = np.array(best_precision_test)

print("Average k: %.2f (+- %.2f)" % (best_neighbors.mean(),best_neighbors.std()))
print("Average mcc (Val): %.2f (+- %.2f)\n" % (best_mcc_val.mean(),best_mcc_val.std()))
print("Average mcc (Test): %.2f (+- %.2f)" % (best_mcc_test.mean(),best_mcc_test.std()))
print("Average auc (Test): %.2f (+- %.2f)" % (best_auc_test.mean(),best_auc_test.std()))
print("Average Precision (Test): %.2f (+- %.2f)" % (best_precision_test.mean(),best_precision_test.std()))
print("Average Recall (Test): %.2f (+- %.2f)" % (best_recall_test.mean(),best_recall_test.std()))


KNN model training

Average k: 1.20 (+- 0.40)
Average mcc (Val): 0.59 (+- 0.03)

Average mcc (Test): 0.54 (+- 0.03)
Average auc (Test): 0.78 (+- 0.02)
Average Precision (Test): 0.65 (+- 0.05)
Average Recall (Test): 0.55 (+- 0.08)


In [6]:
############# ALGORITHM 2: LOGISTIC REGRESSION

print("Logistic regression model training")
print()

list_fpr_lr = []
list_tpr_lr = []
mcc_values_lr = []
auc_values_lr = []
pre_values_lr = []
recall_values_lr = []
acc_values_lr = []

cv = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)


for train_index, val_index in cv.split(X_train,y_train):
    X_train_lr = X_train[train_index]
    X_val = X_train[val_index]
    y_train_lr = y_train[train_index]
    y_val = y_train[val_index]
    
    lr_model = LogisticRegression(penalty="none",solver="lbfgs",max_iter=40000)

    lr_model.fit(X_train_lr, y_train_lr )
    print("Model trained")
    print()

    y_pred_lr = lr_model.predict(X_val)
    y_pred_prob_lr = lr_model.predict_proba(X_val)[:,1]
    
    #compute metrics for ROC Curve
    scores = lr_model.decision_function(X_val)
    fpr, tpr, thresholds = metrics.roc_curve(y_val, scores, drop_intermediate=False)
    
    list_fpr_lr.append(fpr)
    list_tpr_lr.append(tpr)

    acc_v_lr = metrics.accuracy_score(y_val, y_pred_lr)
    auc_v_lr = metrics.roc_auc_score(y_val, y_pred_prob_lr)
    precision_v_lr = metrics.precision_score(y_val, y_pred_lr)
    recall_v_lr = metrics.recall_score(y_val, y_pred_lr)
    mcc_v_lr = metrics.matthews_corrcoef(y_val, y_pred_lr)
    mcc_values_lr.append(mcc_v_lr)
    auc_values_lr.append(auc_v_lr)
    pre_values_lr.append(precision_v_lr)
    recall_values_lr.append(recall_v_lr)
    acc_values_lr.append(acc_v_lr)
    
    
    

print("Validation metrics:")
print("ROC-AUC:\t%.2f (+-%.2f)" % (np.mean(auc_values_lr),np.std(auc_values_lr)))
print("MCC:\t%.2f (+-%.2f)" % (np.mean(mcc_values_lr),np.std(mcc_values_lr)))
print("Precision:\t%.2f (+-%.2f)" % (np.mean(pre_values_lr),np.std(pre_values_lr)))
print("Recall:\t%.2f (+-%.2f)" % (np.mean(recall_values_lr),np.std(recall_values_lr)))
print("ACC:\t%.2f (+-%.2f)" % (np.mean(acc_values_lr),np.std(acc_values_lr)))

print()

y_prediction_lr = lr_model.predict(X_test)
y_prediction_prob_lr = lr_model.predict_proba(X_test)[:,1]

acc_t_lr = metrics.accuracy_score(y_test, y_prediction_lr)
auc_t_lr = metrics.roc_auc_score(y_test, y_prediction_prob_lr)
precision_t_lr = metrics.precision_score(y_test, y_prediction_lr)
recall_t_lr = metrics.recall_score(y_test, y_prediction_lr)
mcc_t_lr = metrics.matthews_corrcoef(y_test, y_prediction_lr)

print("Test metrics:")
print("Accuracy (Test):\t\t%.2f" % acc_t_lr)
print("ROC-AUC (Test):\t\t%.2f" % auc_t_lr)
print("Precision (Test):\t%.2f" % precision_t_lr)
print("Recall (Test):\t\t%.2f" % recall_t_lr)
print("MCC (Test):\t\t%.2f" % mcc_t_lr)
print()

Logistic regression model training

Model trained

Model trained

Model trained

Model trained

Model trained

Model trained

Model trained

Model trained

Model trained

Model trained

Validation metrics:
ROC-AUC:	0.86 (+-0.02)
MCC:	0.40 (+-0.05)
Precision:	0.60 (+-0.05)
Recall:	0.34 (+-0.06)
ACC:	0.90 (+-0.01)

Test metrics:
Accuracy (Test):		0.89
ROC-AUC (Test):		0.88
Precision (Test):	0.61
Recall (Test):		0.30
MCC (Test):		0.38



In [7]:
############# ALGORITHM 3: SVM W/ RBF

start = time.time()

print("Support vector machines model training")

MCC_score = make_scorer(metrics.matthews_corrcoef)

tuned_params = {"C": np.logspace(0,2,4),
               "gamma": np.logspace(-4,-2,4)}

cv = StratifiedKFold(n_splits=10,shuffle=True)

list_fpr_svm_rbf = []
list_tpr_svm_rbf = []
precision_values= []
auc_values_svm_rbf = []
recall_values = []
accuracy_values = []
mcc_values = []

#run a simple SVM
for train_index, val_index in cv.split(X_train,y_train):
    #split data
    X_train_tm = X_train[train_index]
    X_val_tm = X_train[val_index]
    y_train_tm = y_train[train_index]
    y_val_tm = y_train[val_index]
    
    
    #train model
    svc = SVC(kernel='rbf',class_weight='balanced')                #class_weight adapted from Ben-Hur et al. (2008) suggestion
    svm_rbf_model = GridSearchCV(svc, tuned_params, cv=2, scoring=MCC_score)
    svm_rbf_model.fit(X_train_tm, y_train_tm)
    
    print("split completed in (seconds):")
    mid = time.time()
    print(mid - start)
    
    #compute metrics
    scores = svm_rbf_model.decision_function(X_val_tm)
    fpr, tpr, thresholds = metrics.roc_curve(y_val_tm, scores, drop_intermediate=False)
    
    list_fpr_svm_rbf.append(fpr)
    list_tpr_svm_rbf.append(tpr)
    
    y_pred = svm_rbf_model.predict(X_val_tm)
    auc_values_svm_rbf.append(metrics.roc_auc_score(y_val_tm, y_pred))
    accuracy_values.append(metrics.accuracy_score(y_val_tm, y_pred))
    recall_values.append(metrics.recall_score(y_val_tm, y_pred))
    precision_values.append(metrics.precision_score(y_val_tm, y_pred))
    mcc_values.append(metrics.matthews_corrcoef(y_val_tm, y_pred))
    
    #print responses
    
    print("Best parameters set found on development set:")              ####Adapted from SKLearn example: https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html
    print()
    print(svm_rbf_model.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = svm_rbf_model.cv_results_["mean_test_score"]
    stds = svm_rbf_model.cv_results_["std_test_score"]
    for mean, std, params in zip(means, stds, svm_rbf_model.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
             % (mean, std *  2, params))


#print results
print()
print("RBF SVM: Validation data")
print("ROC-AUC:\t%.2f (+-%.2f)" % (np.mean(auc_values_svm_rbf),np.std(auc_values_svm_rbf)))
print("Accuracy:\t%.2f (+-%.2f)" % (np.mean(accuracy_values),np.std(accuracy_values)))
print("Recall:\t\t%.2f (+-%.2f)" % (np.mean(recall_values),np.std(recall_values)))
print("Precision:\t%.2f (+-%.2f)" % (np.mean(precision_values),np.std(precision_values)))
print("MCC:\t\t%.2f (+-%.2f)" % (np.mean(mcc_values),np.std(mcc_values)))
print()

print("RBF SVM: Test data")
#check on test data
y_prediction_svm_rbf = svm_rbf_model.predict(X_test)

#compute metrics
acc_svm_rbf = metrics.accuracy_score(y_test, y_prediction_svm_rbf)
pre_svm_rbf = metrics.precision_score(y_test, y_prediction_svm_rbf)
rec_svm_rbf = metrics.recall_score(y_test, y_prediction_svm_rbf)
mcc_svm_rbf = metrics.matthews_corrcoef(y_test, y_prediction_svm_rbf)

#print results
print("Accuracy (Test):\t%.2f" % acc_svm_rbf)
print("Precision (Test):\t%.2f" % pre_svm_rbf)
print("Recall (Test):\t\t%.2f" % rec_svm_rbf)
print("MCC (Test):\t\t%.2f" % mcc_svm_rbf)


print()
end = time.time()
end1 = (end - start)
print("Total time (seconds):%f" % end1)

Support vector machines model training
split completed in (seconds):
651.0936236381531
Best parameters set found on development set:

{'C': 100.0, 'gamma': 0.002154434690031882}

Grid scores on development set:

0.342 (+/-0.023) for {'C': 1.0, 'gamma': 0.0001}
0.398 (+/-0.018) for {'C': 1.0, 'gamma': 0.00046415888336127773}
0.485 (+/-0.015) for {'C': 1.0, 'gamma': 0.002154434690031882}
0.522 (+/-0.011) for {'C': 1.0, 'gamma': 0.01}
0.386 (+/-0.020) for {'C': 4.641588833612778, 'gamma': 0.0001}
0.447 (+/-0.007) for {'C': 4.641588833612778, 'gamma': 0.00046415888336127773}
0.531 (+/-0.000) for {'C': 4.641588833612778, 'gamma': 0.002154434690031882}
0.535 (+/-0.008) for {'C': 4.641588833612778, 'gamma': 0.01}
0.420 (+/-0.018) for {'C': 21.544346900318832, 'gamma': 0.0001}
0.480 (+/-0.004) for {'C': 21.544346900318832, 'gamma': 0.00046415888336127773}
0.537 (+/-0.021) for {'C': 21.544346900318832, 'gamma': 0.002154434690031882}
0.535 (+/-0.014) for {'C': 21.544346900318832, 'gamma': 0.01}


split completed in (seconds):
6027.832015275955
Best parameters set found on development set:

{'C': 21.544346900318832, 'gamma': 0.01}

Grid scores on development set:

0.346 (+/-0.041) for {'C': 1.0, 'gamma': 0.0001}
0.387 (+/-0.021) for {'C': 1.0, 'gamma': 0.00046415888336127773}
0.480 (+/-0.009) for {'C': 1.0, 'gamma': 0.002154434690031882}
0.540 (+/-0.008) for {'C': 1.0, 'gamma': 0.01}
0.382 (+/-0.025) for {'C': 4.641588833612778, 'gamma': 0.0001}
0.430 (+/-0.006) for {'C': 4.641588833612778, 'gamma': 0.00046415888336127773}
0.538 (+/-0.004) for {'C': 4.641588833612778, 'gamma': 0.002154434690031882}
0.546 (+/-0.003) for {'C': 4.641588833612778, 'gamma': 0.01}
0.409 (+/-0.015) for {'C': 21.544346900318832, 'gamma': 0.0001}
0.474 (+/-0.035) for {'C': 21.544346900318832, 'gamma': 0.00046415888336127773}
0.546 (+/-0.027) for {'C': 21.544346900318832, 'gamma': 0.002154434690031882}
0.548 (+/-0.009) for {'C': 21.544346900318832, 'gamma': 0.01}
0.435 (+/-0.013) for {'C': 100.0, 'gamma':

In [7]:
############# ALGORITHM 3: SVM W/ RBF, best params

svm_rbf_model_ref = SVC(C=21.5443469, kernel='rbf', gamma=.005, class_weight='balanced')
svm_rbf_model_ref.fit(X_train, y_train)


ref_prediction_svm_rbf = svm_rbf_model_ref.predict(X_test)

#compute metrics
acc_svm_rbf_ref = metrics.accuracy_score(y_test, ref_prediction_svm_rbf)
pre_svm_rbf_ref = metrics.precision_score(y_test, ref_prediction_svm_rbf)
rec_svm_rbf_ref = metrics.recall_score(y_test, ref_prediction_svm_rbf)
mcc_svm_rbf_ref = metrics.matthews_corrcoef(y_test, ref_prediction_svm_rbf)

#print results
print("Accuracy (Test):\t%.2f" % acc_svm_rbf_ref)
print("Precision (Test):\t%.2f" % pre_svm_rbf_ref)
print("Recall (Test):\t\t%.2f" % rec_svm_rbf_ref)
print("MCC (Test):\t\t%.2f" % mcc_svm_rbf_ref)


Accuracy (Test):	0.92
Precision (Test):	0.71
Recall (Test):		0.59
MCC (Test):		0.60


In [10]:
############# ALGORITHM #4: SVM W/ TANIMOTO

start = time.time()

print("Support vector machines model training")

MCC_score = make_scorer(metrics.matthews_corrcoef)

tuned_params = {"C": np.logspace(0,3,15)}

cv = StratifiedKFold(n_splits=10,shuffle=True)

list_fpr_svm_tan = []
list_tpr_svm_tan = []
precision_values = []
auc_values_svm_tan = []
recall_values = []
accuracy_values = []
mcc_values = []

#run a simple SVM
for train_index, val_index in cv.split(X_train,y_train):
    #split data
    X_train_tm = X_train[train_index]
    X_val_tm = X_train[val_index]
    y_train_tm = y_train[train_index]
    y_val_tm = y_train[val_index]
    
    
    #train model
    svc = SVC(kernel=Tanimoto(),class_weight='balanced')
    svm_tan_model = GridSearchCV(svc, tuned_params, cv=2, scoring=MCC_score)
    svm_tan_model.fit(X_train_tm, y_train_tm)
    
    print("split completed in (seconds):")
    mid = time.time()
    print(mid - start)
    
    #compute metrics
    scores = svm_tan_model.decision_function(X_val_tm)
    fpr, tpr, thresholds = metrics.roc_curve(y_val_tm, scores, drop_intermediate=False)
    
    list_fpr_svm_tan.append(fpr)
    list_tpr_svm_tan.append(tpr)
    
    y_pred = svm_tan_model.predict(X_val_tm)
    auc_values_svm_tan.append(metrics.roc_auc_score(y_val_tm, y_pred))
    accuracy_values.append(metrics.accuracy_score(y_val_tm, y_pred))
    recall_values.append(metrics.recall_score(y_val_tm, y_pred))
    precision_values.append(metrics.precision_score(y_val_tm, y_pred))
    mcc_values.append(metrics.matthews_corrcoef(y_val_tm, y_pred))
    
    #print responses
    
    print("Best parameters set found on development set:")
    print()
    print(svm_tan_model.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = svm_tan_model.cv_results_["mean_test_score"]
    stds = svm_tan_model.cv_results_["std_test_score"]
    for mean, std, params in zip(means, stds, svm_tan_model.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
             % (mean, std *  2, params))


#print results
print()
print("Tanimoto SVM: Validation data")
print("ROC-AUC:\t%.2f (+-%.2f)" % (np.mean(auc_values_svm_tan),np.std(auc_values_svm_tan)))
print("Accuracy:\t%.2f (+-%.2f)" % (np.mean(accuracy_values),np.std(accuracy_values)))
print("Recall:\t\t%.2f (+-%.2f)" % (np.mean(recall_values),np.std(recall_values)))
print("Precision:\t%.2f (+-%.2f)" % (np.mean(precision_values),np.std(precision_values)))
print("MCC:\t\t%.2f (+-%.2f)" % (np.mean(mcc_values),np.std(mcc_values)))
print()
print("Tanimoto SVM: Test data")

#check on test data
y_prediction_svm_tan = svm_tan_model.predict(X_test)

#compute metrics
acc_svm_tan = metrics.accuracy_score(y_test, y_prediction_svm_tan)
pre_svm_tan = metrics.precision_score(y_test, y_prediction_svm_tan)
rec_svm_tan = metrics.recall_score(y_test, y_prediction_svm_tan)
mcc_svm_tan = metrics.matthews_corrcoef(y_test, y_prediction_svm_tan)

#print results
print("Accuracy (Test):\t%.2f" % acc_svm_tan)
print("Precision (Test):\t%.2f" % pre_svm_tan)
print("Recall (Test):\t\t%.2f" % rec_svm_tan)
print("MCC (Test):\t\t%.2f" % mcc_svm_tan)


print()
end = time.time()
end1 = (end - start)
print("Total time (seconds):%f" % end1)
print()

Support vector machines model training
split completed in (seconds):
79.05540108680725
Best parameters set found on development set:

{'C': 11.787686347935873}

Grid scores on development set:

0.503 (+/-0.015) for {'C': 1.0}
0.529 (+/-0.024) for {'C': 1.637893706954064}
0.540 (+/-0.023) for {'C': 2.6826957952797255}
0.551 (+/-0.018) for {'C': 4.39397056076079}
0.553 (+/-0.040) for {'C': 7.196856730011519}
0.558 (+/-0.035) for {'C': 11.787686347935873}
0.554 (+/-0.044) for {'C': 19.306977288832496}
0.555 (+/-0.048) for {'C': 31.622776601683793}
0.550 (+/-0.034) for {'C': 51.7947467923121}
0.550 (+/-0.034) for {'C': 84.83428982440716}
0.542 (+/-0.028) for {'C': 138.94954943731375}
0.543 (+/-0.026) for {'C': 227.58459260747887}
0.543 (+/-0.028) for {'C': 372.7593720314938}
0.542 (+/-0.030) for {'C': 610.5402296585327}
0.540 (+/-0.032) for {'C': 1000.0}
split completed in (seconds):
163.53880620002747
Best parameters set found on development set:

{'C': 11.787686347935873}

Grid scores on

Accuracy (Test):	0.92
Precision (Test):	0.69
Recall (Test):		0.64
MCC (Test):		0.62

Total time (seconds):861.443737



In [8]:
############# ALGORITHM 4: SVM W/ TANIMOTO, best params

svm_tan_model_ref = SVC(C=50, kernel=Tanimoto(), class_weight='balanced')
svm_tan_model_ref.fit(X_train, y_train)


ref_prediction_svm_tan = svm_tan_model_ref.predict(X_test)

#compute metrics
acc_svm_tan_ref = metrics.accuracy_score(y_test, ref_prediction_svm_tan)
pre_svm_tan_ref = metrics.precision_score(y_test, ref_prediction_svm_tan)
rec_svm_tan_ref = metrics.recall_score(y_test, ref_prediction_svm_tan)
mcc_svm_tan_ref = metrics.matthews_corrcoef(y_test, ref_prediction_svm_tan)

#print results
print("Accuracy (Test):\t%.2f" % acc_svm_tan_ref)
print("Precision (Test):\t%.2f" % pre_svm_tan_ref)
print("Recall (Test):\t\t%.2f" % rec_svm_tan_ref)
print("MCC (Test):\t\t%.2f" % mcc_svm_tan_ref)


Accuracy (Test):	0.92
Precision (Test):	0.69
Recall (Test):		0.62
MCC (Test):		0.61


In [22]:
############# TEST ENSEMBLE

print("Test Ensemble")         #Idea from Mayr et al. (2016); Further insight in Raschka & Mirjalili (2019)
print()

##### PRINT NON-ENSEMBLE MCCs
print("KNN Results:")
print("Average mcc (Test): %.2f (+- %.2f)" % (best_mcc_test.mean(),best_mcc_test.std()))
print()

print("Logistic Regression Results:")
print("MCC (Test):\t\t%.2f" % mcc_t_lr)
print()

print("SVM (RBF) Results:")
print("MCC (Test):\t\t%.2f" % mcc_svm_rbf_ref)
print()

print("SVM (Tanimoto) Results:")
print("MCC (Test):\t\t%.2f" % mcc_svm_tan_ref)
print()
print("------------------------------------------------------------------")
print()
test_prediction_knn = knn_model.predict(X_test)
test_prediction_lr = lr_model.predict(X_test)
test_prediction_svm_tan = svm_tan_model_ref.predict(X_test)
test_prediction_svm_rbf = svm_rbf_model_ref.predict(X_test)

print("Test Ensembles Generating")
print()

test_ensemble1 = []
n = len(test_prediction_knn)
for i in range(n):
    test_ensemble1.append((test_prediction_knn[i] + test_prediction_lr[i] + test_prediction_svm_tan[i])/3)

test_ensemble1 = np.around(test_ensemble1, decimals = 0)

pre_ens1 = metrics.precision_score(y_test, test_ensemble1)
rec_ens1 = metrics.recall_score(y_test, test_ensemble1)
mcc_ens1 = metrics.matthews_corrcoef(y_test, test_ensemble1)


print("Test Ensemble 1 (KNN, LR, SVM Tanimoto) Results:")
print("Precision:\t%.2f" % pre_ens1)
print("Recall:\t\t%.2f" % rec_ens1)
print("MCC:\t\t%.2f" % mcc_ens1)
print()

test_ensemble2 = []
n = len(test_prediction_knn)
for i in range(n):
    test_ensemble2.append((test_prediction_knn[i] + test_prediction_lr[i] + test_prediction_svm_rbf[i])/3)

test_ensemble2 = np.around(test_ensemble2, decimals = 0)

pre_ens2 = metrics.precision_score(y_test, test_ensemble2)
rec_ens2 = metrics.recall_score(y_test, test_ensemble2)
mcc_ens2 = metrics.matthews_corrcoef(y_test, test_ensemble2)


print("Test Ensemble 2 (KNN, LR, SVM RBF) Results:")
print("Precision:\t%.2f" % pre_ens2)
print("Recall:\t\t%.2f" % rec_ens2)
print("MCC:\t\t%.2f" % mcc_ens2)
print()

test_ensemble3 = []
n = len(test_prediction_knn)
for i in range(n):
    test_ensemble3.append((test_prediction_knn[i] + test_prediction_svm_tan[i] + test_prediction_svm_rbf[i])/3)

test_ensemble3 = np.around(test_ensemble3, decimals = 0)

pre_ens3 = metrics.precision_score(y_test, test_ensemble3)
rec_ens3 = metrics.recall_score(y_test, test_ensemble3)
mcc_ens3 = metrics.matthews_corrcoef(y_test, test_ensemble3)


print("Test Ensemble 3 (KNN, SVM RBF, SVM Tanimoto) Results:")
print("Precision:\t%.2f" % pre_ens3)
print("Recall:\t\t%.2f" % rec_ens3)
print("MCC:\t\t%.2f" % mcc_ens3)
print()

test_ensemble4 = []
n = len(test_prediction_knn)
for i in range(n):
    test_ensemble4.append((test_prediction_lr[i] + test_prediction_svm_tan[i] + test_prediction_svm_rbf[i])/3)

test_ensemble4 = np.around(test_ensemble4, decimals = 0)

pre_ens4 = metrics.precision_score(y_test, test_ensemble4)
rec_ens4 = metrics.recall_score(y_test, test_ensemble4)
mcc_ens4 = metrics.matthews_corrcoef(y_test, test_ensemble4)


print("Test Ensemble 4 (LR, SVM RBF, SVM Tanimoto) Results:")
print("Precision:\t%.2f" % pre_ens4)
print("Recall:\t\t%.2f" % rec_ens4)
print("MCC:\t\t%.2f" % mcc_ens4)
print()

test_ensemble5 = []
n = len(test_prediction_knn)
for i in range(n):
    test_ensemble5.append(((0.16*test_prediction_lr[i]) + (0.35*test_prediction_svm_tan[i]) + 
                           (0.3*test_prediction_svm_rbf[i]) + (0.25*test_prediction_knn[i])))

test_ensemble5 = np.around(test_ensemble5, decimals = 0)

pre_ens5 = metrics.precision_score(y_test, test_ensemble5)
rec_ens5 = metrics.recall_score(y_test, test_ensemble5)
mcc_ens5 = metrics.matthews_corrcoef(y_test, test_ensemble5)


print("Test Ensemble 5 (LR, KNN, SVM RBF, SVM Tanimoto) Results:")
print("Precision:\t%.2f" % pre_ens5)
print("Recall:\t\t%.2f" % rec_ens5)
print("MCC:\t\t%.2f" % mcc_ens5)
print()


Test Ensemble

KNN Results:
Average mcc (Test): 0.54 (+- 0.03)

Logistic Regression Results:
MCC (Test):		0.38

SVM (RBF) Results:
MCC (Test):		0.60

SVM (Tanimoto) Results:
MCC (Test):		0.61

------------------------------------------------------------------

Test Ensembles Generating

Test Ensemble 1 (KNN, LR, SVM Tanimoto) Results:
Precision:	0.76
Recall:		0.56
MCC:		0.62

Test Ensemble 2 (KNN, LR, SVM RBF) Results:
Precision:	0.75
Recall:		0.54
MCC:		0.60

Test Ensemble 3 (KNN, SVM RBF, SVM Tanimoto) Results:
Precision:	0.73
Recall:		0.61
MCC:		0.62

Test Ensemble 4 (LR, SVM RBF, SVM Tanimoto) Results:
Precision:	0.72
Recall:		0.60
MCC:		0.61

Test Ensemble 5 (LR, KNN, SVM RBF, SVM Tanimoto) Results:
Precision:	0.73
Recall:		0.62
MCC:		0.63



In [23]:
#####Guess Ensemble

print("Ensemble generating")
print()

prediction_knn = knn_model.predict(unknowns)
prediction_lr = lr_model.predict(unknowns)
prediction_svm_rbf = svm_rbf_model_ref.predict(unknowns)
prediction_svm_tan = svm_tan_model_ref.predict(unknowns)

count_knn = np.count_nonzero(prediction_knn)
count_lr = np.count_nonzero(prediction_lr)
count_svm_rbf = np.count_nonzero(prediction_svm_rbf)
count_svm_tan = np.count_nonzero(prediction_svm_tan)

print("Initial toxic counts:")
print("K-nearest neighbor = ", count_knn)
print("Logistic regression = ", count_lr)
print("Support vector machines (rbf) = ", count_svm_rbf)
print("Support vector machines (tanimoto) = ", count_svm_tan)
print()

ensemble = []
n = len(prediction_knn)
for i in range(n):
    ensemble.append(((0.16*prediction_lr[i]) + (0.35*prediction_svm_tan[i]) + 
                           (0.3*prediction_svm_rbf[i]) + (0.25*prediction_knn[i])))

ensemble = np.around(ensemble, decimals = 0)
count_ensemble = np.count_nonzero(ensemble)

print("Ensemble generated")
print("Ensemble count = ", count_ensemble)

Ensemble generating

Initial toxic counts:
K-nearest neighbor =  66
Logistic regression =  55
Support vector machines (rbf) =  56
Support vector machines (tanimoto) =  60

Ensemble generated
Ensemble count =  57


In [24]:
prediction = np.column_stack((sample_ids_unknowns, ensemble))
np.savetxt("Ensemble.csv", prediction, delimiter=",", fmt='%s')

print("Guess file generated")
print("Good luck")

Guess file generated
Good luck
